In [7]:
from dotenv import load_dotenv
import sys, os
import re, random, pandas as pd
from collections import defaultdict

In [8]:

# Load environment variables from .env file
load_dotenv()

# Get the API key
GENIUS_CLIENT_ACCESS_TOKEN = os.getenv("GENIUS_API_KEY")
SPOTIPY_CLIENT_ID = os.getenv("CLIENT_ID")
SPOTIPY_CLIENT_SECRET = os.getenv("CLIENT_SECRET")


# Use the API key
print(f"Your API key is: {GENIUS_CLIENT_ACCESS_TOKEN}")

Your API key is: xysLEfXPULlltSqXKiL1raqCdAaCybJeHKa0vLWl6Hz9lQTkb3D1GVGue0vCNoue


In [9]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import lyricsgenius
from spotipy.oauth2 import SpotifyClientCredentials

# Spotify API credentials
client_credentials_manager = SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET)
# Initialize the Spotify API client
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout=40) #spotify object to access API

# Create a Genius API client
genius = lyricsgenius.Genius(GENIUS_CLIENT_ACCESS_TOKEN, timeout=40)

In [16]:
# This function is to get lyrics
def lyrics(song, artist):
    # genius = lyricsgenius.Genius('')
    song = genius.search_song(title=song, artist=artist)
    try:
      lyrics = song.lyrics
      print(lyrics)
      print('\n')
    except AttributeError:
      print("There are no lyrics for this song on Genius")
      print('\n')

In [19]:

song = "Die with a smile"
artist = "Bruno Mars, Lady Gaga"

lyrics(song, artist)


Searching for "Die with a smile" by Bruno Mars, Lady Gaga...
Done.
186 ContributorsTranslationsTürkçeEspañolPortuguêsDeutschFrançaisItalianoالعربيةΕλληνικάTiếng ViệtBahasa IndonesiaفارسیNederlands中文ไทย / Phasa ThaiРусскийPolskiМакедонски日本語HebrewУкраїнськаСрпскиČeskyසිංහලDie With A Smile Lyrics“Die With A Smile” was the comeback single by Lady Gaga and Bruno Mars, who both had not released music in two to three years. The song was surprise announced a day before the release on August 15, and… Read More [Intro: Bruno Mars]
(Ooh, ooh)

[Verse 1: Bruno Mars]
I, I just woke up from a dream
Where you and I had to say goodbye
And I don't know what it all means
But since I survived, I realized

[Pre-Chorus: Bruno Mars]
Wherever you go, that's where I'll follow
Nobody's promised tomorrow
So I'ma love you every night like it's the last night
Like it's the last night

[Chorus: Bruno Mars]
If the world was ending
I'd wanna be next to you
If the party was over
And our time on Earth was through
I'd

In [20]:
def scrape_lyrics(artistname, songname):
    artistname2 = str(artistname.replace(' ', '-')) if ' ' in artistname else str(artistname)
    songname2 = str(songname.replace(' ', '-')) if ' ' in songname else str(songname)
    page = requests.get('https://genius.com/' + artistname2 + '-' + songname2 + '-' + 'lyrics')
    html = BeautifulSoup(page.text, 'html.parser')
    lyrics1 = html.find("div", class_="lyrics")
    lyrics2 = html.find("div", class_="Lyrics__Container-sc-1ynbvzw-2 jgQsqn")
    if lyrics1:
        lyrics = lyrics1.get_text()
    elif lyrics2:
        lyrics = lyrics2.get_text()
    elif lyrics1 == lyrics2 == None:
        lyrics = None
    return lyrics


In [21]:
def lyrics_onto_frame(df1, artist_name):
    for i, x in enumerate(df1['Track Name']):
        test = scrape_lyrics(artist_name, x)
        print(test)
        df1.loc[i, 'Lyrics'] = test
    return df1

In [47]:
# ------------------------------------------------------------------
# Build the “Student_dataset” – grab ≈400 candidate tracks, then keep 200 random rows
# ------------------------------------------------------------------

target_genre   = "Reggae"   # ← change if you pick another extra-genre
songs_required = 150
batch          = 50        # Spotify’s per-call limit
max_offset     = 1000      # crawl first 1000 results (0‒950)


In [48]:
# Parallel lists → final CSV columns
artist_names, track_names, release_dates, genres, lyrics = [], [], [], [], []

# Keep (artist, track) pairs so we never double-count
seen = set()

# Shuffle offsets so we sample *random* slices of Spotify’s catalogue
offsets = list(range(0, max_offset, batch))
random.shuffle(offsets)

In [49]:
for offset in offsets:
    if len(lyrics) >= songs_required:
        break

    pop_tracks = sp.search(q=f'genre:"{target_genre}"', type='track',
                           limit=batch, offset=offset)

    for track in pop_tracks['tracks']['items']:
        if len(lyrics) >= songs_required:
            break

        key = (track['artists'][0]['name'].lower(), track['name'].lower())
        if key in seen:
            continue                               # skip duplicates
        seen.add(key)

        try:
            # ---------- Genius lyrics ----------
            song = genius.search_song(track['name'], track['artists'][0]['name'])
            if not song:
                continue
            l = song.lyrics
            l = re.sub(r'^.*?\n+',  '', l, count=1)   # drop header
            l = re.sub(r'\s+',      ' ', l).strip()   # single-line text
            if len(l) < 200:                          # too short → skip
                continue

            # ---------- Append to dataset ----------
            lyrics.append(l)
            artist_names.append(track['artists'][0]['name'])
            track_names.append(track['name'])
            release_dates.append(track['album']['release_date'][:4])  # keep YYYY
            genres.append(target_genre)

        except Exception:
            continue


Searching for "Solida" by Almighty...
Done.
Searching for "Senja Teduh Pelita" by MALIQ & D'Essentials...
Done.
Searching for "EL HECHIZO" by Ovy On The Drums...
Done.
Searching for "Na' Personal" by Miky Woodz...
Done.
Searching for "Parcerita" by Myke Towers...
Done.
Searching for "G Low Kitty" by El Bogueto...
Done.
Searching for "Mojando Asientos (feat. Feid)" by Maluma...
Done.
Searching for "Ñero Session 23" by Alu Mix...
No results found for: 'Ñero Session 23 Alu Mix'
Searching for "Na Szczycie" by Grubson...
Done.
Searching for "YO AK" by Feid...
Done.
Searching for "Aventura" by Luny Tunes...
Done.
Searching for "Hay Algo en Ti" by Luny Tunes...
Done.
Searching for "Esta Noche" by Mike Bahía...
Done.
Searching for "Naturaleza" by Anuel AA...
Done.
Searching for "EL RITMO QUE NOS UNE - feat. Selección Colombia" by Ryan Castro...
Done.
Searching for "Kung Ika'y Akin" by Chocolate Factory...
Done.
Searching for "Knees" by Ocean Alley...
Done.
Searching for "Rap É Compromisso" by 

In [50]:
# ------------------------------------------------------------------
# Build DataFrame, randomly down-sample to exactly 200, write CSV
# ------------------------------------------------------------------
df = pd.DataFrame({
    "artist_name":   artist_names,
    "track_name":    track_names,
    "release_date":  release_dates,
    "genre":         genres,
    "lyrics":        lyrics
}).sample(n=songs_required, random_state=42)


In [51]:
df

,artist_name,track_name,release_date,genre,lyrics
73,Anuel AA,Hasta Que Dios Diga,2020,Reggae,"[Intro: Anuel AA] Brr Hoy la noche se acaba, t..."
18,Bryant Myers,Gan-Ga - Remix,2019,Reggae,[Intro: Anuel AA & Bryant Myers] Brr Lanalizer...
118,Ele A El Dominio,Codeine Crazy - Spanish Remix,2019,Reggae,"[Intro] 'Toy bien loco Ajá Ah, ah Overdose Com..."
78,Yannc,Sigue Bailandome,2018,Reggae,"[Intro: Darkiel, Myke Towers & Brray] (Sheesh)..."
76,Hale,The Day You Said Goodnight,2005,Reggae,Take me as you are Push me off the road The sa...
...,...,...,...,...,...
71,Ozuna,El Farsante,2017,Reggae,"[Intro] Oh-oh-oh ¡Yeah, eh! [Pre-Coro] Extraño..."
106,Camilo,Autodiagnóstico,2024,Reggae,[Verso 1] Ya fui al doctor seis veces Y me rep...
14,Chocolate Factory,Kung Ika'y Akin,2019,Reggae,Hindi naman ako nagmamadali Natatakot lang ako...
92,FloyyMenor,Gata Only (Remix),2024,Reggae,"[Intro: Ozuna & FloyyMenor] (Mami, te siento l..."


In [52]:
#Save the df to .csv

In [54]:
df.to_csv("data/Student_dataset.csv", index=False, encoding="utf-8")
print("✅  Student_dataset.csv created with shape:", df.shape)

✅  Student_dataset.csv created with shape: (150, 5)
